In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
## 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
## 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
## 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
## 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
## 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
region = 'daqing'

In [5]:
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = ee.Image(img).clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# RF训练

In [6]:
### 将人工选取样本的影像变成影像集
## 导入人工采集的样本
manual_samples = ee.FeatureCollection('users/311605001111/manual_samples/DAQING_2020merge')
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))

image_id = ee.List(manual_samples.distinct('Image_id').aggregate_array('Image_id'))
image_set = ee.List([])
for i in image_id.getInfo():
    if 'LT05' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    elif 'LE07' in i:
        image_set = image_set.add(ee.Image(str(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
    else:
        image_set = image_set.add(ee.Image(str(i)).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))
# 图像集，image_col
image_col = ee.ImageCollection(image_set)
print(image_col.aggregate_array('system:id').getInfo())

manual-sample number:2052
['LANDSAT/LE07/C01/T1_SR/LE07_119028_20200425', 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20200924', 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20201010', 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20201111', 'LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526', 'LANDSAT/LC08/C01/T1_SR/LC08_120028_20200611']


In [7]:
## 导入自动采集的样本
samples = ee.FeatureCollection('users/311605001111/SAMPLE_DAQING/' + region + '_kmean100_1620')
print('auto-sample number:{}'.format(samples.size().getInfo()))

label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# # cluster与waterclass的对比
# def Compare(feature):
#     cluster = ee.Number(feature.get('cluster'))
#     waterclass = ee.Number(feature.get('waterclass'))
#     ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
#     return ft
# # k-mean聚类
# def k_mean(sampleSET):
#     clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
#     result = sampleSET.cluster(clusterer)
#     right = result.map(Compare).filter(ee.Filter.eq('eq',0))
#     error = result.map(Compare).filter(ee.Filter.eq('eq',1))
#     filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
#     return filtered_sample

# auto_samples = k_mean(samples)
# print("筛选后，剩余的样本数： ",auto_samples.size().getInfo())
auto_samples = samples
trainedClassifier = ee.Classifier.smileRandomForest(150).train(auto_samples,label,bands)

auto-sample number:21838


# 精度验证函数

In [8]:
def image_accuracy(img):
    image = maskSR(water_index(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    manual_point = manual_samples.filter(ee.Filter.eq('Image_id',img.get('system:id'))).map(lambda i : i.centroid())
    inter_points = image.sampleRegions(**{
        'collection': manual_point,
        'properties': ['label'],
        'scale': 30,
        'geometries': True,
    })
    inter_samples = inter_points.map(lambda i : i.setMulti({'Image_id':img.get('system:id')}))
    return inter_samples

samples_dataset = image_col.map(image_accuracy).flatten()

In [9]:
# 利用误差矩阵进行验证
water = samples_dataset.filter(ee.Filter.eq('label',1)).randomColumn('random').sort('random').limit(200)
no_water = samples_dataset.filter(ee.Filter.eq('label',0)).randomColumn('random').sort('random').limit(800)
com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()
test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

test_accuracy = samples_dataset.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

[[792, 8], [1, 199]]
total accuracy:0.991
kappa:0.972239358420728
[[1347, 22], [3, 678]]
total accuracy:0.9878048780487805
kappa:0.9727052432561979


In [10]:
# 利用误差矩阵进行验证
diff_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
diff_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
cor_water = samples_dataset.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',1)).randomColumn('random').sort('random').limit(200-diff_water.size().getInfo())
cor_nowater = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',0)).randomColumn('random').sort('random').limit(800-diff_nowater.size().getInfo())
total_samples = ee.FeatureCollection([diff_water,diff_nowater,cor_water,cor_nowater]).flatten()

test_accuracy = total_samples.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

[[778, 22], [3, 197]]
total accuracy:0.975
kappa:0.9245624622812311


# 样本分布

In [ ]:
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filter(ee.Filter.lt('CLOUD_COVER',30)) \
       .filterDate('2020-01-01', '2020-12-31')          
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l8.size().getInfo())

In [ ]:
refer_img = l8.map(maskSR).median().clip(roi).visualize(**{'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4,'opacity':0.65})
manual_samples = ee.FeatureCollection('users/311605001111/PRD_2020').map(lambda i : i.centroid())
print('manual-sample number:{}'.format(manual_samples.size().getInfo()))

blend = refer_img.blend(manual_samples.filter(ee.Filter.eq('label',0)).draw('red')).blend(manual_samples.filter(ee.Filter.eq('label',1)).draw('yellow'))
Map.addLayer(blend, {}, "Blend")

In [ ]:
# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
# # 大庆市
# region = [123.9355, 45.8362,125.3055, 46.8762]
# # 天津市
# region = [115.9244, 38.6272,117.4944,39.6372]
# # 武汉市
# region = [113.7393, 29.8642,115.0993, 30.9242]
# # 珠三角
region = [112.7614, 22.2347,114.0514, 23.2547]
# 青藏高原
# region = [89.8826, 34.6579,91.3626, 35.8279]
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(blend, region=region)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'Region D', fontsize=28)

# 附录

In [ ]:
# 图像
img = water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122033_20200218'))
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(img.clip(roi),visParams,'img')

# 分类图像
image = maskSR(water_index(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
# Map.addLayer(image,{'palette':['white','blue'],'min':0,'max':1},'image')
Map.addLayer(image.selfMask(),{'palette':['blue']},'image')
# 点
a = ee.Geometry.Point([117.00910929827164, 38.890901875718455])
vis_point = {'color': 'yellow', 'pointSize': 100,'fillColor': 'yellow'}
Map.addLayer(a,vis_point,'a')

In [ ]:
miss = samples_dataset.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
print(miss.size().getInfo())
print(miss.getInfo())
print(miss.distinct('Image_id').aggregate_array('Image_id').getInfo())